<a href="https://colab.research.google.com/github/DarkNightSoldier/DTApp/blob/main/ProyectoBD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Extracción de información del PDF de Historia Académica

In [583]:
!pip install PyPDF2


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [584]:
import re
from PyPDF2 import PdfReader
import json

# Cargar el archivo PDF
pdf_path = "HistoriaAcadem.pdf"
reader = PdfReader(pdf_path)

# Extraer texto de todas las páginas
text = ""
for page in reader.pages:
    text += page.extract_text()

# Buscar las secciones requeridas usando expresiones regulares
nombre = re.search(r"Nombre:\s+(.*)", text).group(1)
documento = re.search(r"Documento:\s+(\d+)", text).group(1)
plan = re.search(r"Plan:\s+(.*?)(?:\s+Hist\.|$)", text).group(1)
papa_promedio = re.search(r"(\d+,\d+)\s+\(Acumulado\)\s+PREGRADO\s+P\.A\.P\.A", text).group(1)
creditos_dt = re.search(r"Créditos de estudio doble titulación\s+(\d+)", text).group(1)
porcentaje_avance = re.search(r"Porcentaje de avance\s+(\d+\.?\d*)%", text).group(1)

# Imprimir resultados
print("Nombre:", nombre)
print("Documento:", documento)
print("Plan:", plan)
print("Créditos de Estudio Doble Titulación:", creditos_dt)
print("porcentaje de avance:", porcentaje_avance)

# Información por semestre
periodos = text.split("PERIODO")
# Array que contiene la información de asignaturas cursadas:
asignaturas_cursadas = []

# Imprimir cada sección del periodo
for i, periodo in enumerate(periodos):
    texto_periodo = periodo.strip()
    periodo_match = re.search(r"(\d{4}-\d{1}S)", texto_periodo)
    periodo = periodo_match.group(1) if periodo_match else "No especificado"

    # Patrón para extraer las asignaturas con calificación únicamente numérica
    asignatura_pattern = r"(?P<Nom_Asignatura>.+?)\s+\((?P<Cod_Asignatura>\d+)\).+?(?P<Creditos>\d+).+?Fund\. Obligatoria\s+\(B\)\s+(?P<Calificacion>\d+,\d+)"

    # Buscar todas las asignaturas en el texto del periodo
    asignaturas = re.finditer(asignatura_pattern, texto_periodo)

    # Añadir a la lista de asignaturas cursadas
    for asignatura in asignaturas:
        asignaturas_cursadas.append({
            "Periodo": periodo,
            "Cod_Asignatura": asignatura.group("Cod_Asignatura"),
            "Nom_Asignatura": asignatura.group("Nom_Asignatura").strip(),
            "Creditos": asignatura.group("Creditos"),
            "Calificacion": asignatura.group("Calificacion")
        })

# Convertir resultados a JSON
resultados_json = json.dumps(asignaturas_cursadas, indent=4, ensure_ascii=False)

# Imprimir JSON
print(resultados_json)

Nombre: Pepito Perez Daza
Documento: 123456789
Plan: Matemáticas (2518)
Créditos de Estudio Doble Titulación: 70
porcentaje de avance: 20.1
[
    {
        "Periodo": "2022-1S",
        "Cod_Asignatura": "2016377",
        "Nom_Asignatura": "Cálculo diferencial en una variable",
        "Creditos": "4",
        "Calificacion": "3,0"
    },
    {
        "Periodo": "2022-1S",
        "Cod_Asignatura": "2015168",
        "Nom_Asignatura": "Fundamentos de matemáticas",
        "Creditos": "4",
        "Calificacion": "3,3"
    },
    {
        "Periodo": "2022-2S",
        "Cod_Asignatura": "2016375",
        "Nom_Asignatura": "Programación orientada a objetos",
        "Creditos": "3",
        "Calificacion": "4,2"
    },
    {
        "Periodo": "2022-2S",
        "Cod_Asignatura": "2015556",
        "Nom_Asignatura": "Cálculo integral en una variable",
        "Creditos": "4",
        "Calificacion": "3,0"
    },
    {
        "Periodo": "2022-2S",
        "Cod_Asignatura": "2015555",


## 2. Estudio a partir de la historia extraida.

In [585]:
import pandas as pd
# Conversión del JSON/Diccionario a DataFrame
asignaturas_estudiante = pd.DataFrame(asignaturas_cursadas)
asignaturas_estudiante

,Periodo,Cod_Asignatura,Nom_Asignatura,Creditos,Calificacion
0,2022-1S,2016377,Cálculo diferencial en una variable,4,"3,0"
1,2022-1S,2015168,Fundamentos de matemáticas,4,"3,3"
2,2022-2S,2016375,Programación orientada a objetos,3,"4,2"
3,2022-2S,2015556,Cálculo integral en una variable,4,"3,0"
4,2022-2S,2015555,Álgebra lineal básica,4,"3,3"
5,2022-2S,2015181,Sistemas numéricos,4,"3,5"


In [586]:
asignaturas_estudiante = pd.read_csv("asignaturas_ejemplo2.csv",sep=";")
asignaturas_estudiante

,Periodo,Cod_Asignatura,Nom_Asignatura,Creditos,Calificacion
0,2018-2S,2015172,Geometría Elemental,4,3
1,2018-2S,2016377,Cálculo diferencial en una variable,4,"3,7"
2,2018-2S,2015168,Fundamentos de Matemáticas,4,"3,5"
3,2019-1S,2015556,Cálculo integral en una variable,4,"4,2"
4,2019-1S,2015555,Álgebra lineal básica,4,4
5,2019-1S,2015181,Sistemas numéricos,4,"3,1"
6,2019-2S,2025819,Introducción a la teoría de conjuntos,4,"4,6"
7,2019-2S,2015180,Programación y métodos numéricos,4,"4,5"
8,2019-2S,2016342,Cálculo de Ecuaciones Diferenciales Ordinaria,4,"3,2"
9,2019-2S,2015162,Cálculo vectorial,4,"4,1"


In [587]:
# Conexión a la base de datos SQLite
import sqlite3
Apuntador = sqlite3.connect('DatosApp.db')
cursor = Apuntador.cursor()

In [588]:
!pip install pandasql
from pandasql import sqldf
import pandas as pd


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [589]:
# Queremos obtener una lista de EQUIVALENCIAS CANDIDATAS

def Generar_Lista_Candidatas(asignaturas_estudiante):
    # Query para obtener las asignaturas que cursó y que pertenecen al programa 
    # Esto es Equivalencias con igual código
    lista_asignaturas_cursadas = tuple(asignaturas_estudiante["Cod_Asignatura"])
    query_1 = f"""
        SELECT 
            Cod_Asignatura_CC AS Código, 
            Nom_Agrupacion AS Agrupación,
            Tipo 
        FROM Asignaturas_CC 
        WHERE Cod_Asignatura_CC IN {lista_asignaturas_cursadas}"""
    asignaturas_iguales = pd.read_sql_query(query_1, Apuntador)

    # Join para completar la información de las asignaturas
    query_2 = """ 
        SELECT 
            asignaturas_estudiante.Periodo AS "Periodo Académico",
            asignaturas_estudiante.Cod_Asignatura AS "Código",
            asignaturas_estudiante.Nom_Asignatura AS "Asignatura",
            asignaturas_iguales.Código AS "Código_CC",
            asignaturas_estudiante.Nom_Asignatura AS "Asignatura_CC",
            asignaturas_iguales.Agrupación AS "Agrupación",
            asignaturas_estudiante.Calificacion AS "Nota",
            asignaturas_iguales.Tipo AS "Tipo",
            asignaturas_estudiante.Creditos AS "Créditos"
        FROM asignaturas_estudiante
        JOIN asignaturas_iguales
        ON asignaturas_iguales.Código = asignaturas_estudiante.Cod_Asignatura;
    """

    equivalencias_candidatas = pd.DataFrame(sqldf(query_2))


    # Equivalencias con diferente código
    # Obtener la equivalencia desde la base de datos
    query_3 = f"""
        SELECT 
            Cod_Asignatura AS Código, 
            Cod_Asignatura_CC AS 'Código_CC'
        FROM Equivalencias
        WHERE Cod_Asignatura IN {lista_asignaturas_cursadas}"""
    asignaturas_equivalentes = pd.read_sql_query(query_3, Apuntador)

    # Join para obtener la agrupación y tipo de las asignaturas equivalentes
    query_4 = f"""
        SELECT 
            Cod_Asignatura_CC AS 'Código_CC', 
            Nom_Agrupacion AS Agrupación,
            Tipo 
        FROM Asignaturas_CC 
        WHERE Cod_Asignatura_CC IN {tuple(asignaturas_equivalentes["Código_CC"])}"""
    info_equivalentes = pd.read_sql_query(query_4, Apuntador)

    # Join para obtener el número de créditos y nombre de las asignaturas equivalentes
    query_5 = f"""
        SELECT 
            Cod_Asignatura AS Código_CC, 
            Nom_Asignatura AS Asignatura_CC,
            Creditos
        FROM Asignaturas_Info 
        WHERE Asignaturas_Info IN {tuple(asignaturas_equivalentes["Código_CC"])}"""
    info_equivalentes_2 = pd.read_sql_query(query_5, Apuntador)

    # Join para completar la información de las asignaturas
    query_6 = """
    SELECT 
        e.Periodo AS "Periodo Académico",
        e.Cod_Asignatura AS "Código",
        e.Nom_Asignatura AS "Asignatura",
        a.Código_CC AS "Código_CC",
        ie2.Asignatura_CC AS "Asignatura_CC",
        ie.Agrupación AS "Agrupación",
        ie2.Creditos AS "Créditos",
        ie.Tipo AS "Tipo"
    FROM asignaturas_estudiante AS e
    JOIN asignaturas_equivalentes AS a 
    ON e.Cod_Asignatura = a.Código
    JOIN info_equivalentes AS ie 
    ON a.Código_CC = ie.Código_CC
    JOIN info_equivalentes_2 AS ie2  
    ON ie.Código_CC = ie2.Código_CC;
    """
    # Concatenar las equivalencias de asignaturas iguales con las de diferentes
    equivalencias_candidatas = pd.concat([equivalencias_candidatas,pd.DataFrame(sqldf(query_6))],axis=0)
    return equivalencias_candidatas

lista_candidatas = Generar_Lista_Candidatas(asignaturas_estudiante)
display(lista_candidatas)

,Periodo Académico,Código,Asignatura,Código_CC,Asignatura_CC,Agrupación,Nota,Tipo,Créditos
0,2018-2S,2016377,Cálculo diferencial en una variable,2016377,Cálculo diferencial en una variable,MATEMÁTICAS Y ESTRUCTURAS DISCRETAS,"3,7",B,4
1,2018-2S,2015168,Fundamentos de Matemáticas,2015168,Fundamentos de Matemáticas,MATEMÁTICAS Y ESTRUCTURAS DISCRETAS,"3,5",B,4
2,2019-1S,2015556,Cálculo integral en una variable,2015556,Cálculo integral en una variable,MATEMÁTICAS Y ESTRUCTURAS DISCRETAS,"4,2",B,4
3,2019-1S,2015555,Álgebra lineal básica,2015555,Álgebra lineal básica,MATEMÁTICAS Y ESTRUCTURAS DISCRETAS,4,B,4
4,2019-1S,2015181,Sistemas numéricos,2015181,Sistemas numéricos,MATEMÁTICAS Y ESTRUCTURAS DISCRETAS,"3,1",B,4
5,2019-2S,2025819,Introducción a la teoría de conjuntos,2025819,Introducción a la teoría de conjuntos,MATEMÁTICAS Y ESTRUCTURAS DISCRETAS,"4,6",B,4
6,2019-2S,2016342,Cálculo de Ecuaciones Diferenciales Ordinaria,2016342,Cálculo de Ecuaciones Diferenciales Ordinaria,MATEMÁTICAS Y ESTRUCTURAS DISCRETAS,"3,2",B,4
7,2019-2S,2015162,Cálculo vectorial,2015162,Cálculo vectorial,MATEMÁTICAS Y ESTRUCTURAS DISCRETAS,"4,1",B,4
8,2020-1S,2015152,Grupos y anillos,2015152,Grupos y anillos,MATEMÁTICAS Y ESTRUCTURAS DISCRETAS,"4,2",O,4
9,2020-1S,2026555,Álgebra abstracta y computacional,2026555,Álgebra abstracta y computacional,ALGORITMOS Y TEORÍA DE LA COMPUTACIÓN,"4,6",C,4


In [590]:
asignaturas_estudiante

,Periodo,Cod_Asignatura,Nom_Asignatura,Creditos,Calificacion
0,2018-2S,2015172,Geometría Elemental,4,3
1,2018-2S,2016377,Cálculo diferencial en una variable,4,"3,7"
2,2018-2S,2015168,Fundamentos de Matemáticas,4,"3,5"
3,2019-1S,2015556,Cálculo integral en una variable,4,"4,2"
4,2019-1S,2015555,Álgebra lineal básica,4,4
5,2019-1S,2015181,Sistemas numéricos,4,"3,1"
6,2019-2S,2025819,Introducción a la teoría de conjuntos,4,"4,6"
7,2019-2S,2015180,Programación y métodos numéricos,4,"4,5"
8,2019-2S,2016342,Cálculo de Ecuaciones Diferenciales Ordinaria,4,"3,2"
9,2019-2S,2015162,Cálculo vectorial,4,"4,1"


In [591]:
# Consulta de agrupaciones
query_7 = 'SELECT * FROM Agrupaciones_CC'
agrupaciones = pd.read_sql_query(query_7, Apuntador)
agrupaciones

# Cursadas en Fundamentación (Por el momento únicamente las obligatorias)
cursadas_fundamentacion = lista_candidatas[lista_candidatas["Tipo"] == "B"]

# Pendientes en Fundamentación - Obligatoria B
query_8 = f"""
SELECT
    Cod_Asignatura_CC AS 'Código',
    Nom_Agrupacion AS 'Agrupación'
FROM Asignaturas_CC
WHERE Cod_Asignatura_CC NOT IN {tuple(cursadas_fundamentacion["Código_CC"])} AND Tipo = 'B'
"""
pendientes_b = pd.read_sql_query(query_8, Apuntador)

query_9 = f"""
SELECT
    Cod_Asignatura as 'Código',
    Nom_Asignatura as 'Asignatura',
    Creditos as 'Créditos'
FROM Asignaturas_Info
WHERE Cod_Asignatura IN {tuple(pendientes_b["Código"])}
"""
pendientes_b_info = pd.read_sql_query(query_9, Apuntador)
pendientes_b_info 

query_10 = f"""
SELECT
    p.Agrupación AS 'Agrupación',
    i.Código as 'Código',
    i.Asignatura as 'Asignatura',
    i.Créditos as 'Créditos'
FROM pendientes_b_info as i
JOIN pendientes_b as p 
ON i.Código = p.Código
"""
tabla_pendientes_b = pd.DataFrame(sqldf(query_10)).sort_values(by="Agrupación")

# Calcular la suma de créditos por agrupación
suma_creditos = tabla_pendientes_b.groupby("Agrupación")["Créditos"].sum().reset_index()
suma_creditos.rename(columns={"Créditos": "Créditos pendientes por cursar de la agrupación"}, inplace=True)

# Unir esta información al DataFrame original
tabla_pendientes_b = tabla_pendientes_b.merge(suma_creditos, on="Agrupación", how="left")

# Pendientes en Disciplinar - Obligatoria C y Trabajo de Grado P
# Cursadas en Fundamentación (Por el momento únicamente las obligatorias)
cursadas_disciplinar = lista_candidatas[lista_candidatas["Tipo"] == "C"]

# Agrupación de fundamentación
lista_agrup_fund = agrupaciones[agrupaciones["Tipologia"]=="Disciplinar"]["Nom_Agrupacion"]

query_11 = f"""
SELECT
    Cod_Asignatura_CC AS 'Código',
    Nom_Agrupacion AS 'Agrupación'
FROM Asignaturas_CC
WHERE Cod_Asignatura_CC NOT IN {tuple(cursadas_fundamentacion["Código_CC"])} AND Tipo = 'C'
"""
pendientes_c = pd.read_sql_query(query_11, Apuntador)

query_12 = f"""
SELECT
    Cod_Asignatura as 'Código',
    Nom_Asignatura as 'Asignatura',
    Creditos as 'Créditos'
FROM Asignaturas_Info
WHERE Cod_Asignatura IN {tuple(pendientes_c["Código"])}
"""
pendientes_c_info = pd.read_sql_query(query_12, Apuntador)
pendientes_c_info 

query_13 = f"""
SELECT
    p.Agrupación AS 'Agrupación',
    i.Código as 'Código',
    i.Asignatura as 'Asignatura',
    i.Créditos as 'Créditos'
FROM pendientes_c_info as i
JOIN pendientes_c as p 
ON i.Código = p.Código
"""
tabla_pendientes_c = pd.DataFrame(sqldf(query_13)).sort_values(by="Agrupación")

# Calcular la suma de créditos por agrupación
suma_creditos = tabla_pendientes_c.groupby("Agrupación")["Créditos"].sum().reset_index()
suma_creditos.rename(columns={"Créditos": "Créditos pendientes por cursar de la agrupación"}, inplace=True)

# Unir esta información al DataFrame original
tabla_pendientes_c = tabla_pendientes_c.merge(suma_creditos, on="Agrupación", how="left")


# Pendientes en Disciplinar - Obligatoria P Trabajo de Grado
query_14 = f"""
SELECT
    a.Nom_Agrupacion AS 'Agrupación',
    a.Cod_Asignatura_CC AS 'Código',
    i.Nom_Asignatura AS 'Asignatura',
    i.Creditos AS 'Créditos',
    agrup.Cant_Optativos AS 'Créditos pendientes por cursar de la agrupación'
FROM Asignaturas_CC AS a
JOIN Asignaturas_Info AS i
ON i.Cod_Asignatura = a.Cod_Asignatura_CC
JOIN Agrupaciones_CC AS agrup
ON agrup.Nom_Agrupacion = a.Nom_Agrupacion
WHERE a.Nom_Agrupacion = 'TRABAJO DE GRADO'
"""
tabla_pendientes_disciplinar = pd.concat([tabla_pendientes_c,pd.read_sql_query(query_14, Apuntador)]).reset_index(drop=True)

# Contador de créditos Optativas O y T
creditos_contados = pd.DataFrame(columns=["Agrupación","Tipo","Créditos aprobados","Créditos excedentes"])

# Seleccionar en Fundamentación Optativa - O las asignaturas a equivaler
# Agrupación de fundamentación
lista_agrup_fund = agrupaciones[agrupaciones["Tipologia"]=="Fundamentación"]["Nom_Agrupacion"]

# Seleccionar las asignaturas con mayor nota hasta completar la agrupación
for agrupacion in lista_agrup_fund:
    cantidad_creditos_o = agrupaciones[agrupaciones["Nom_Agrupacion"]==agrupacion]["Cant_Optativos"].values[0]
    asignaturas = lista_candidatas[(lista_candidatas["Agrupación"] == agrupacion) & (lista_candidatas["Tipo"] == "O")].sort_values(by="Nota", ascending=False).reset_index(drop=True)
    suma_creditos = 0
    i = 0
    while suma_creditos < cantidad_creditos_o and i<len(asignaturas):
        # Seleccionar el primer registro - el que tiene mejor nota
        asignatura = asignaturas.iloc[0]  

        # Agregarlo a cursadas_fundamentacion
        cursadas_fundamentacion = pd.concat([cursadas_fundamentacion, pd.DataFrame([asignatura])], ignore_index=True)

        # Eliminar el primer registro de tabla
        asignaturas = asignaturas.iloc[1:].reset_index(drop=True)

        # Incrementar el contador
        i += 1
    creditos_contados = pd.concat([creditos_contados, pd.DataFrame([[agrupacion,"Fundamentación", suma_creditos, max(suma_creditos-cantidad_creditos_o,0)]], columns=["Agrupación", "Tipo", "Créditos aprobados","Créditos excedentes"])], ignore_index=True)


# Seleccionar en Disciplinar - T las asignaturas a equivaler
# Agrupación de Disciplinar
lista_agrup_disciplinar = agrupaciones[agrupaciones["Tipologia"]=="Disciplinar"]["Nom_Agrupacion"]

# Seleccionar las asignaturas con mayor nota hasta completar la agrupación
for agrupacion in lista_agrup_disciplinar:
    cantidad_creditos_t = agrupaciones[agrupaciones["Nom_Agrupacion"]==agrupacion]["Cant_Optativos"].values[0]
    asignaturas = lista_candidatas[(lista_candidatas["Agrupación"] == agrupacion) & (lista_candidatas["Tipo"] == "T")].sort_values(by="Nota", ascending=False).reset_index(drop=True)
    suma_creditos = 0
    i = 0
    while suma_creditos < cantidad_creditos_t and i<len(asignaturas):
        # Seleccionar el primer registro - el que tiene mejor nota
        asignatura = asignaturas.iloc[0]  

        # Agregarlo a cursadas_fundamentacion
        cursadas_disciplinar = pd.concat([cursadas_disciplinar, pd.DataFrame([asignatura])], ignore_index=True)

        # Eliminar el primer registro de tabla
        asignaturas = asignaturas.iloc[1:].reset_index(drop=True)

        # Incrementar el contador
        i += 1
    creditos_contados = pd.concat([creditos_contados, pd.DataFrame([[agrupacion, "Disciplinar", suma_creditos, max(suma_creditos-cantidad_creditos_t,0)]], columns=["Agrupación","Tipo","Créditos aprobados","Créditos excedentes"])], ignore_index=True)

# Seleccionar en Libre Elección - L las asignaturas a equivaler
    

# Visualizar pendientes en Fundamentación Optativa O
# Pendientes en Fundamentación - Optativa O
pendientes_o = agrupaciones[agrupaciones["Tipologia"]=="Fundamentación"][["Nom_Agrupacion", "Cant_Optativos"]]
pendientes_o.rename(columns={"Nom_Agrupacion": "Agrupación", "Cant_Optativos": "Créditos exigidos en la agrupación"}, inplace=True)
pendientes_o["Créditos pendientes por cursar de la agrupación"] = (
    pendientes_o["Créditos exigidos en la agrupación"] - 
    creditos_contados[creditos_contados["Agrupación"].isin(pendientes_o["Agrupación"])]["Créditos aprobados"].values
).clip(lower=0)

# Visualizar pendientes en Disciplinar Optativa T
# Pendientes en Disciplinar - Optativa T
pendientes_t = agrupaciones[agrupaciones["Tipologia"]=="Disciplinar"][["Nom_Agrupacion", "Cant_Optativos"]]
pendientes_t.rename(columns={"Nom_Agrupacion": "Agrupación", "Cant_Optativos": "Créditos exigidos en la agrupación"}, inplace=True)
pendientes_t["Créditos pendientes por cursar de la agrupación"] = (
    pendientes_t["Créditos exigidos en la agrupación"] - 
    creditos_contados[creditos_contados["Agrupación"].isin(pendientes_t["Agrupación"])]["Créditos aprobados"].values
).clip(lower=0)

# Seleccionar en Libre Elección - L las asignaturas a equivaler
candidatas_libre = asignaturas_estudiante[~(asignaturas_estudiante["Cod_Asignatura"].isin(cursadas_fundamentacion["Código"]) | asignaturas_estudiante["Cod_Asignatura"].isin(cursadas_disciplinar["Código"]))]
# Ordenar por calificación
candidatas_libre = candidatas_libre.sort_values(by="Calificacion", ascending=False).reset_index(drop=True)

# Seleccionar en Libre Elección - L las asignaturas a equivaler
creditos_libre = agrupaciones[agrupaciones["Tipologia"]=="Libre Elección"]["Cant_Optativos"].values[0]
cursadas_libre = pd.DataFrame(columns=candidatas_libre.columns)
suma_creditos = 0
i = 0
while suma_creditos<creditos_libre and i<len(candidatas_libre):
    # Seleccionar el primer registro - el que tiene mejor nota
    asignatura = candidatas_libre.iloc[0]  

    # Agregarlo a cursadas_fundamentacion
    cursadas_libre = pd.concat([cursadas_libre, pd.DataFrame([asignatura])], ignore_index=True)

    # Eliminar el primer registro de tabla
    candidatas_libre = candidatas_libre.iloc[1:].reset_index(drop=True)

    # Incrementar el contador
    i += 1

#Guardar dato de los excedentes
creditos_contados = pd.concat([creditos_contados, pd.DataFrame([["LIBRE ELECCIÓN", "Libre Elección", suma_creditos, max(suma_creditos-creditos_libre,0)]], columns=["Agrupación","Tipo","Créditos aprobados","Créditos excedentes"])], ignore_index=True)
                              
libre_cursadas = pd.DataFrame(columns=["Periodo Académico", "Código", "Asignatura", "Código_2", "Asignatura_2", "Agrupación", "Nota", "Créditos"])
libre_cursadas["Periodo Académico"] = cursadas_libre["Periodo"]
libre_cursadas["Código"] = cursadas_libre["Cod_Asignatura"]
libre_cursadas["Asignatura"] = cursadas_libre["Nom_Asignatura"]
libre_cursadas["Código_2"] = cursadas_libre["Cod_Asignatura"]
libre_cursadas["Asignatura_2"] = cursadas_libre["Nom_Asignatura"]
libre_cursadas["Nota"] = cursadas_libre["Calificacion"]
libre_cursadas["Créditos"] = cursadas_libre["Creditos"]

#Tabla de Resumen General
resumen_general = pd.DataFrame(columns=["Créditos","Fund. Obl. B","Fund. Opt. O","Disc. Obl. C","Disc. Opt. T","Trabajo de Grado P","Libre Elección L", "Total"])
resumen_general["Créditos"] = ["Exigidos","Convalidados/equivalentes","Pendientes","Excedentes"]

resumen_general.loc[resumen_general["Créditos"] == "Exigidos", "Fund. Obl. B"] = agrupaciones[agrupaciones["Tipologia"]=="Fundamentación"]["Cant_Obligatorios"].sum()
resumen_general.loc[resumen_general["Créditos"] == "Exigidos", "Fund. Opt. O"] = agrupaciones[agrupaciones["Tipologia"]=="Fundamentación"]["Cant_Optativos"].sum()
resumen_general.loc[resumen_general["Créditos"] == "Exigidos", "Disc. Obl. C"] = agrupaciones[agrupaciones["Tipologia"]=="Disciplinar"]["Cant_Obligatorios"].sum()
resumen_general.loc[resumen_general["Créditos"] == "Exigidos", "Disc. Opt. T"] = agrupaciones[agrupaciones["Tipologia"]=="Disciplinar"]["Cant_Optativos"].sum()
resumen_general.loc[resumen_general["Créditos"] == "Exigidos", "Trabajo de Grado P"] = agrupaciones[agrupaciones["Tipologia"]=="Trabajo de grado"]["Cant_Optativos"].sum()
resumen_general.loc[resumen_general["Créditos"] == "Exigidos", "Libre Elección L"] = agrupaciones[agrupaciones["Tipologia"]=="Libre Elección"]["Cant_Optativos"].sum()
resumen_general.loc[resumen_general["Créditos"] == "Exigidos", "Total"] = resumen_general.loc[resumen_general["Créditos"] == "Exigidos", resumen_general.columns[1:]].sum(axis=1)

resumen_general.loc[resumen_general["Créditos"] == "Convalidados/equivalentes", "Fund. Obl. B"] = cursadas_fundamentacion[cursadas_fundamentacion["Tipo"]=="B"]["Créditos"].astype(int).sum()
resumen_general.loc[resumen_general["Créditos"] == "Convalidados/equivalentes", "Fund. Opt. O"] = cursadas_fundamentacion[cursadas_fundamentacion["Tipo"]=="O"]["Créditos"].astype(int).sum()
resumen_general.loc[resumen_general["Créditos"] == "Convalidados/equivalentes", "Disc. Obl. C"] = cursadas_disciplinar[cursadas_disciplinar["Tipo"]=="C"]["Créditos"].astype(int).sum()
resumen_general.loc[resumen_general["Créditos"] == "Convalidados/equivalentes", "Disc. Opt. T"] = cursadas_disciplinar[cursadas_disciplinar["Tipo"]=="T"]["Créditos"].astype(int).sum()
# De acuerdo a la norma el trabajo de grado no es equivalible ni convalidable
resumen_general.loc[resumen_general["Créditos"] == "Convalidados/equivalentes", "Trabajo de Grado P"] = 0
resumen_general.loc[resumen_general["Créditos"] == "Convalidados/equivalentes", "Libre Elección L"] = libre_cursadas["Créditos"].astype(int).sum()
resumen_general.loc[resumen_general["Créditos"] == "Convalidados/equivalentes", "Total"] = resumen_general.loc[resumen_general["Créditos"] == "Convalidados/equivalentes", resumen_general.columns[1:]].sum(axis=1)


resumen_general.loc[resumen_general["Créditos"] == "Pendientes", "Fund. Obl. B"] = tabla_pendientes_b["Créditos"].astype(int).sum()
resumen_general.loc[resumen_general["Créditos"] == "Pendientes", "Disc. Obl. C"] = tabla_pendientes_c["Créditos"].astype(int).sum()
resumen_general.loc[resumen_general["Créditos"] == "Pendientes", "Fund. Opt. O"] = pendientes_o["Créditos pendientes por cursar de la agrupación"].astype(int).sum()
resumen_general.loc[resumen_general["Créditos"] == "Pendientes", "Disc. Opt. T"] = pendientes_t["Créditos pendientes por cursar de la agrupación"].astype(int).sum()
resumen_general.loc[resumen_general["Créditos"] == "Pendientes", "Trabajo de Grado P"] = agrupaciones[agrupaciones["Tipologia"]=="Trabajo de grado"]["Cant_Optativos"].sum()
resumen_general.loc[resumen_general["Créditos"] == "Pendientes", "Libre Elección L"] = max(0,resumen_general.loc[resumen_general["Créditos"] == "Exigidos", "Libre Elección L"].values[0] - resumen_general.loc[resumen_general["Créditos"] == "Convalidados/equivalentes", "Libre Elección L"].values[0])
resumen_general.loc[resumen_general["Créditos"] == "Pendientes", "Total"] = resumen_general.loc[resumen_general["Créditos"] == "Pendientes", resumen_general.columns[1:]].sum(axis=1)

# En las obligatorias nunca se obtienen créditos excedentes
resumen_general.loc[resumen_general["Créditos"] == "Excedentes", "Fund. Obl. B"] = 0
resumen_general.loc[resumen_general["Créditos"] == "Excedentes", "Disc. Obl. C"] = 0
resumen_general.loc[resumen_general["Créditos"] == "Excedentes", "Trabajo de Grado P"] = 0
resumen_general.loc[resumen_general["Créditos"] == "Excedentes", "Fund. Opt. O"] = creditos_contados[creditos_contados["Tipo"]=="Fundamentación"]["Créditos excedentes"].sum()
resumen_general.loc[resumen_general["Créditos"] == "Excedentes", "Disc. Opt. T"] = creditos_contados[creditos_contados["Tipo"]=="Disciplinar"]["Créditos excedentes"].sum()
resumen_general.loc[resumen_general["Créditos"] == "Excedentes", "Libre Elección L"] = creditos_contados[creditos_contados["Tipo"]=="Libre Elección"]["Créditos excedentes"].sum()
resumen_general.loc[resumen_general["Créditos"] == "Excedentes", "Total"] = resumen_general.loc[resumen_general["Créditos"] == "Excedentes", resumen_general.columns[1:]].sum(axis=1)



In [592]:
import warnings
from datetime import datetime
warnings.simplefilter(action='ignore', category=Warning)
# Información inicial
query_15 = f""" 
SELECT *
FROM Planes_de_Estudio"""

info_plan = pd.read_sql_query(query_15, Apuntador)

print("1. Datos Generales")
info_inicial = pd.DataFrame([["Nombre", nombre],
                             ["DNI",documento],
                             ["Plan de estudios origen (1er plan):", plan],
                             ["Plan de estudios doble titulación (2° plan)", str(info_plan["Codigo_plan"].values[0])+" - "+ str(info_plan["Nombre_plan"].values[0])],
                             ["Fecha de la solicitud",datetime.today().strftime("%d/%m/%Y")]])
display(info_inicial)
print("3. Resumen general de créditos del segundo plan de estudios:")
display(resumen_general)
print("4. Cuadro de equivalencia y convalidaciones de asignaturas")

# Formatear últimas 2 columnas según formato interno
print("Componente de FUNDAMENTACIÓN")
cursadas_fundamentacion["Créditos_B"] = cursadas_fundamentacion.apply(lambda row: row["Créditos"] if row["Tipo"] == "B" else "", axis=1)
cursadas_fundamentacion["Créditos_O"] = cursadas_fundamentacion.apply(lambda row: row["Créditos"] if row["Tipo"] == "O" else "", axis=1)
cursadas_fundamentacion.drop(columns=["Tipo", "Créditos"], inplace=True)
display(cursadas_fundamentacion)


print("Componente DISCIPLINAR")
cursadas_disciplinar["Créditos_C"] = cursadas_disciplinar.apply(lambda row: row["Créditos"] if row["Tipo"] == "C" else "", axis=1)
cursadas_disciplinar["Créditos_T"] = cursadas_disciplinar.apply(lambda row: row["Créditos"] if row["Tipo"] == "T" else "", axis=1)
cursadas_disciplinar.drop(columns=["Tipo", "Créditos"], inplace=True)
display(cursadas_disciplinar)

print("Componente de LIBRE ELECCIÓN")
display(libre_cursadas)

# Mostrar el resultado
print("5. Asignaturas pendientes por cursar en el segundo plan de estudios")
print("Pendientes en Fundamentación - Obligatoria B")
display(tabla_pendientes_b)
print("Pendientes en Fundamentación - Optativa O")
display(pendientes_o)
print("Pendientes en Disciplinar - Obligatoria C y P")
display(tabla_pendientes_disciplinar)
print("Pendientes en Fundamentación - Optativa T")
display(pendientes_t)

print("6. El Consejo de Facultad Recomienda / No recomienda")
recomienda = None
razon = ""
if float(porcentaje_avance)<40:
    recomienda = False
    razon = "No dispone de un porcentaje de avance mayor al 40 en el primer plan de estudios."
elif papa_promedio >= 4.3:
    recomienda = True
    razon = "Aunque no dispone de los créditos suficientes para cursar las asignaturas pendientes de aprobación en el segundo plan, presenta un PAPA mayor o igual a 4,3."
elif resumen_general.loc[resumen_general["Créditos"] == "Pendientes", "Total"].values[0] > creditos_dt:
    recomienda = False
    razon = "No dispone de los créditos suficientes para cursar las asignaturas pendientes de aprobación en el segundo plan."
else: 
    recomienda = True
    razon = "Dispone de los créditos suficientes para cursar las asignaturas pendientes de aprobación en el segundo plan de estudios."

print("Recomendación: ", "Recomienda" if recomienda else "No recomienda")
print(razon)

1. Datos Generales


,0,1
0,Nombre,Pepito Perez Daza
1,DNI,123456789
2,Plan de estudios origen (1er plan):,Matemáticas (2518)
3,Plan de estudios doble titulación (2° plan),2933 - Ciencias de la Computación
4,Fecha de la solicitud,12/02/2025


3. Resumen general de créditos del segundo plan de estudios:


,Créditos,Fund. Obl. B,Fund. Opt. O,Disc. Obl. C,Disc. Opt. T,Trabajo de Grado P,Libre Elección L,Total
0,Exigidos,49,11,21,22,8,28,139
1,Convalidados/equivalentes,43,12,12,10,0,28,105
2,Pendientes,6,11,21,22,8,0,68
3,Excedentes,0,0,0,0,0,0,0


4. Cuadro de equivalencia y convalidaciones de asignaturas
Componente de FUNDAMENTACIÓN


...

Pendientes en Disciplinar - Obligatoria C y P


,Agrupación,Código,Asignatura,Créditos,Créditos pendientes por cursar de la agrupación
0,ALGORITMOS Y TEORÍA DE LA COMPUTACIÓN,2015174,Introducción a la teoría de la computación,4,11
1,ALGORITMOS Y TEORÍA DE LA COMPUTACIÓN,2016696,Algoritmos,3,11
2,ALGORITMOS Y TEORÍA DE LA COMPUTACIÓN,2026555,Algebra abstracta y computacional,4,11
3,COMPUTACIÓN CIENTÍFICA,2019072,Análisis numérico I,4,4
4,"SISTEMAS OPERATIVOS, DE CÓMPUTO Y COMPILADORES",2016698,Elementos de computadores,3,6
5,"SISTEMAS OPERATIVOS, DE CÓMPUTO Y COMPILADORES",2016707,Sistemas operativos,3,6
6,TRABAJO DE GRADO,2027633,Trabajo de Grado - Trabajos Investigativos,8,8
7,TRABAJO DE GRADO,2027634,Trabajo de Grado - Asignaturas de Posgrado,8,8
8,TRABAJO DE GRADO,2027636,Trabajo de Grado - Pasantías,8,8


Pendientes en Fundamentación - Optativa T


,Agrupación,Créditos exigidos en la agrupación,Créditos pendientes por cursar de la agrupación
3,ALGORITMOS Y TEORÍA DE LA COMPUTACIÓN,3,3
4,SEGURIDAD INFORMÁTICA Y CODIFICACIÓN,7,7
5,"SISTEMAS OPERATIVOS, DE CÓMPUTO Y COMPILADORES",3,3
6,COMPUTACIÓN CIENTÍFICA,3,3
7,COMPUTACIÓN APLICADA,6,6


6. El Consejo de Facultad Recomienda / No recomienda
Recomendación:  No recomienda
No dispone de un porcentaje de avance mayor al 40 en el primer plan de estudios.


In [593]:
creditos_contados

,Agrupación,Tipo,Créditos aprobados,Créditos excedentes
0,MATEMÁTICAS Y ESTRUCTURAS DISCRETAS,Fundamentación,0,0
1,CIENCIAS NATURALES Y ESTADÍSTICA,Fundamentación,0,0
2,PROGRAMACIÓN Y ESTRUCTURAS DE DATOS,Fundamentación,0,0
3,ALGORITMOS Y TEORÍA DE LA COMPUTACIÓN,Disciplinar,0,0
4,SEGURIDAD INFORMÁTICA Y CODIFICACIÓN,Disciplinar,0,0
5,"SISTEMAS OPERATIVOS, DE CÓMPUTO Y COMPILADORES",Disciplinar,0,0
6,COMPUTACIÓN CIENTÍFICA,Disciplinar,0,0
7,COMPUTACIÓN APLICADA,Disciplinar,0,0
8,LIBRE ELECCIÓN,Libre Elección,0,0
